In [2]:
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

### *Set up Display*

In [3]:
is_python = 'inline' in matplotlib.get_backend()
if is_python: from IPython import display

### *Deep Q-Network*

In [4]:
class DQN(nn.Module):
    
    def __init__(self,img_height,img_width):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features=img_height*img_width*3, out_features=24)
        self.fc2 = nn.Linear(in_features=24, out_features=32)
        self.out = nn.Linear(in_features=32, out_features=2)
    
    def forward(self,t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t

### *Experience Class*

In [5]:
Experience = namedtuple(
    'Experience',
    ('start','action','next_state','reward')
)

### *Replay Memory*

In [6]:
class ReplayMemoy():
    def __init__(self,capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
    
    def push(self,experience):
        if (len(self.memory)) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1
    
    def sample(self,batch_size):
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self,batch_size):
        return len(self.memory) >= batch_size

### Epsilon Greedy Strategy

In [8]:
class EpsilonGreedyStrategy():
    def __init__(self, start,end,decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self,current_step):
        return self.end + (self.start - self.end) * \
                math.exp(-1. * current_step * self.decay)

### *Reinforcement Learning Agent*

In [9]:
class Agent():
    
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device
        
    def select_action(self, state, policy_net):
        rate = self.strategy.get_exploration_rate(self.current_step)
        self.current_step += 1
        
        if rate > random.random():
            action = random.randrange(self.num_actions) # explore
            return torch.tensor([action]).to(device)
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(device) #exploit

### *Environment Manager*

In [ ]:
class CartPoleEnvManager():
    def __init__(self,device):
        self.device = device
        self.env = gym.make('CartPole-v0').unwrapped
        self.env.reset()
        self.current_screen = None
        self.done = False
        
    def reset(self):
        self.env.reset()
        self.current_screen = None
        
    def close(self):
        self.env.close()
        
    def render(self,mode='human'):
        return self.env.render(mode)
    
    def num_actions_available(self):
        return self.env.action_space.n
    
    def take_action(self,action):
        _,reward,self.done,_ = self.env.step(action.item())
        return torch.tensor([reward],device=self.device)
    
    def just_starting(self):
        return self.current_screen is None
    
    def get_state(self):
        if self.just_starting() or self.done :
            self.current_screen = self.get_processed_screen()
            black_screen = torch.zeros_like(self.current_screen)
            return black_screen
        else:
            s1 = self.current_screen
            s2 = self.get_processed_screen()
            self.current_screen = s2
            return s2-s1
    
    def get_screen_height(self):
        screen = self.get_processed_screen()
        return screen.shape[2]
    
    def get_processed_screen(self):
        
        screen = self.render('rgb_array').transpose((2,0,1))
        screen = self.crop_screen(screen)
        return self.transpose_screen_data(screen)
    
    def crop_screen(self,screen):
        screen_height = screen.shape[1]
        
        #Strip of top and bottom
        top = int(screen_height*0.4)
        bottom = int(screen_height*0.8)
        screen = screen[:,top:bottom,:]
        return screen
    
    def transform_screen_data(self,screen):
        #Convert to float, rescale, convert to tensor
        screen = np.ascontiguousarray(screen,dtype=np.float32) / 255
        screen = torch.from_numpy(screen)
        
        # Use torchvision package to compose image transforms
        resize = T.Compose([
            T.ToPILImage(),
            T.Resize((40,90))
        ])
        
        return resize(screen).unsqueeze(0).to(self.device)